In [ ]:
# syft absolute
import syft as sy
from syft import test_settings

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
this_worker_pool_name = "bigquery-pool"

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2

In [ ]:
# !pip list | grep bigquery

In [ ]:
# !pip install db-dtypes google-cloud-bigquery

In [ ]:
# Look up the worker pools and identify the name of the one that has the required packages
# After, bind the endpoint to that workerpool
high_client.worker_pools

In [ ]:
# third party
from apis.bigquery import test_query

mock_func = test_query.make_mock(
    settings={
        "credentials": test_settings.gce_service_account.to_dict(),
        "region": test_settings.gce_region,
        "project_id": test_settings.gce_project_id,
        "CALLS_PER_MIN": 10,
    }
)

In [ ]:
private_func = test_query.make_private(
    settings={
        "credentials": test_settings.gce_service_account.to_dict(),
        "region": test_settings.gce_region,
        "project_id": test_settings.gce_project_id,
    }
)

In [ ]:
new_endpoint = sy.TwinAPIEndpoint(
    path="bigquery.test_query",
    description="This endpoint allows to query Bigquery storage via SQL queries.",
    private_function=private_func,
    mock_function=mock_func,
    worker_pool=this_worker_pool_name,
)

high_client.custom_api.add(endpoint=new_endpoint)

In [ ]:
# Here, we update the endpoint to timeout after 100s (rather the default of 60s)
high_client.api.services.api.update(
    endpoint_path="bigquery.test_query", endpoint_timeout=120
)

In [ ]:
high_client.api.services.api.update(
    endpoint_path="bigquery.test_query", hide_mock_definition=True
)

In [ ]:
# it currently hangs here because the reloaded server in this notebook doesnt recreate the worker consumers

In [ ]:
# Test mock version
result = high_client.api.services.bigquery.test_query.mock(
    sql_query=f"SELECT * FROM {test_settings.dataset_1}.{test_settings.table_1} LIMIT 10"
)
result

In [ ]:
assert len(result) == 10

In [ ]:
# third party
from apis.bigquery import schema

schema_function = schema.make_schema(
    settings={
        "credentials": test_settings.gce_service_account.to_dict(),
        "region": test_settings.gce_region,
        "project_id": test_settings.gce_project_id,
        "dataset_1": test_settings.dataset_1,
        "table_1": test_settings.table_1,
        "table_2": test_settings.table_2,
        "CALLS_PER_MIN": 5,
    },
    worker_pool=this_worker_pool_name,
)

In [ ]:
high_client.custom_api.add(endpoint=schema_function)
high_client.refresh()

In [ ]:
# third party
from apis.bigquery import submit_query

submit_query_function = submit_query.make_submit_query(
    settings={}, worker_pool=this_worker_pool_name
)

In [ ]:
high_client.custom_api.add(endpoint=submit_query_function)

In [ ]:
high_client.api.services.api.update(
    endpoint_path="bigquery.submit_query", hide_mock_definition=True
)

In [ ]:
high_client.custom_api.api_endpoints()

In [ ]:
assert len(high_client.custom_api.api_endpoints()) == 3

In [ ]:
high_client.api.services.bigquery.test_query

In [ ]:
high_client.api.services.bigquery.submit_query

In [ ]:
# Test mock version
result = high_client.api.services.bigquery.test_query.mock(
    sql_query=f"SELECT * FROM {test_settings.dataset_1}.{test_settings.table_1} LIMIT 10"
)
result

In [ ]:
assert len(result) == 10

In [ ]:
# Test private version
result = high_client.api.services.bigquery.test_query.private(
    sql_query=f"SELECT * FROM {test_settings.dataset_1}.{test_settings.table_1} LIMIT 10"
)
result

In [ ]:
assert len(result) == 10

In [ ]:
# todo can we clean up the duplicate exception messages?

In [ ]:
# Test mock version for wrong queries
with sy.raises(sy.SyftException(public_message="*must be qualified with a dataset*")):
    _ = high_client.api.services.bigquery.test_query.mock(
        sql_query="SELECT * FROM invalid_table LIMIT 1"
    )

In [ ]:
# Test private version
result = high_client.api.services.bigquery.test_query.private(
    sql_query=f"SELECT * FROM {test_settings.dataset_1}.{test_settings.table_1} LIMIT 1"
)
result

In [ ]:
assert len(result) == 1

In [ ]:
# Inspect the context state on an endpoint
state = high_client.api.services.bigquery.test_query.mock.context.state
state

In [ ]:
# todo fix catch exceptions in code eval so that state can be written

In [ ]:
assert len(state["info@openmined.org"]) >= 3

In [ ]:
# Testing schema
result = high_client.api.services.bigquery.schema()
result

In [ ]:
assert len(result) == 23

In [ ]:
# Testing submit query
result = high_client.api.services.bigquery.submit_query(
    func_name="my_func",
    query=f"SELECT * FROM {test_settings.dataset_1}.{test_settings.table_1} LIMIT 1",
)

In [ ]:
result

In [ ]:
assert "Query submitted" in result

In [ ]:
server.land()